# import module

In [1]:
import os
import re
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer, PreTrainedTokenizerFast, BertTokenizerFast
import pandas as pd
import random
from tensorflow import keras
import tqdm.notebook
import time
import pickle
# import Ipy
from IPython.display import clear_output

# load Model, tokenzier

In [11]:
# model = TFGPT2LMHeadModel.from_pretrained("taeminlee/kogpt2", from_pt=True)
tokenizer = PreTrainedTokenizerFast.from_pretrained("taeminlee/kogpt2")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


# preprocessing

In [4]:
from IPython.display import clear_output
DATA_PATH = '../../3.data/BERT/'
FILE_NAME = 'JobKor_SplitedSentence_final.pickle'

f = open(DATA_PATH + FILE_NAME, 'rb')
data = pickle.load(f)
f.close()

train_data = []
count4 = 0
count3 = 0

for contents in data:
    for content in contents:
        for paragraph in content:
            # 3문장 데이터 생성
            if len(paragraph) == 3:
                count3 += 1
                dataTemp = ''
                dataTemp = "</s><s>".join(paragraph)
                dataTemp = '<s>' + dataTemp + '</s>'
                train_data.append(dataTemp)
#             print("데이터 개수: 총 {}개, 4문장 {}개, 3문장 {}개".format(len(train_data), count4, count3), end='\r', flush=True)
            
            # 3문장 데이터 생성
            elif len(paragraph) == 4:
                for i, sentence in enumerate(paragraph[:-2]):
                    count3 += 1
                    dataTemp = ''
                    dataTemp = "</s><s>".join([sentence, paragraph[i+1], paragraph[i+2]])
                    dataTemp = '<s>' + dataTemp + '</s>'
                    train_data.append(dataTemp)
#             print("데이터 개수: 총 {}개, 4문장 {}개, 3문장 {}개".format(len(train_data), count4, count3), end='\r', flush=True)
                
            # 4문장 데이터 생성
            elif len(paragraph) > 4:
                for i, sentence in enumerate(paragraph[:-3]):
                    count4 += 1
                    dataTemp = ''
                    dataTemp = "</s><s>".join([sentence, paragraph[i+1], paragraph[i+2], paragraph[i+3]])
                    dataTemp = '<s>' + dataTemp + '</s>'
                    train_data.append(dataTemp)
        print("데이터 개수: 총 {}개, 4문장 {}개, 3문장 {}개".format(len(train_data), count4, count3), end='\r', flush=True)
    
clear_output(wait=False)
print("데이터 개수: 총 {}개, 4문장 {}개, 3문장 {}개".format(len(train_data), count4, count3), end='\r', flush=True)
train_data[:10]

['<s>새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다.</s><s>조금 늦은 나이의 도전이라 부족한 영어 실력으로 많은 어려움에 부딪쳤습니다.</s><s>잠깐 좌절도 했지만 그보다 가장 힘들었던 것은 부모님이 주신 소중한 기회를 제대로 활용하고 있지 못하고 있는 저의 모습이었습니다.</s><s>저는 좌절보다는 노력을 택하였습니다.</s>',
 '<s>조금 늦은 나이의 도전이라 부족한 영어 실력으로 많은 어려움에 부딪쳤습니다.</s><s>잠깐 좌절도 했지만 그보다 가장 힘들었던 것은 부모님이 주신 소중한 기회를 제대로 활용하고 있지 못하고 있는 저의 모습이었습니다.</s><s>저는 좌절보다는 노력을 택하였습니다.</s><s>매주 영어 학원과 스터디그룹에 참석하였으며 다양한 이들과의 대화를 위해 혼자 펍에 가 친구를 사귀기도 하고 아르바이트를 통해 많은 이들과의 소통으로 영어 실력뿐만 아니라 소통력과 관계력까지도 키울 수 있었습니다.</s>',
 '<s>잠깐 좌절도 했지만 그보다 가장 힘들었던 것은 부모님이 주신 소중한 기회를 제대로 활용하고 있지 못하고 있는 저의 모습이었습니다.</s><s>저는 좌절보다는 노력을 택하였습니다.</s><s>매주 영어 학원과 스터디그룹에 참석하였으며 다양한 이들과의 대화를 위해 혼자 펍에 가 친구를 사귀기도 하고 아르바이트를 통해 많은 이들과의 소통으로 영어 실력뿐만 아니라 소통력과 관계력까지도 키울 수 있었습니다.</s><s>이러한 경험을 바탕으로 경동나비엔의 일원이 되어서도 어떠한 어려움 속에서도 좌절하지 않고 백절불굴의 자세로 부족한 점을 계속해서 연구하고 개선하여 끊임없는 변화 속에서 끊임없는 노력으로 성공이라는 꽃을 피우도록 하겠습니다.</s>',
 '<s>코카콜라의 뉴코크 실패는 소비자는 단순히 제품만을 소비하는 것이 아니라 그 브랜드 즉, 제품의 상징성을 구매하는 심리를 보여주고 있습니다.</s><s>해외시장에 존재하는 고객은 국내시장에 존재하는 고객들보다 기업에 대한 적은 정보를 가지고 있으며 시장진출

In [12]:
train_data_encode = [tokenizer.encode(d) for d in train_data]

In [13]:
inputs = []
labels = []

for encode_d in train_data_encode:
    inputs.append(encode_d[:-1])
    labels.append(encode_d[1:])

In [14]:
tokenizer.decode(inputs[0])

'<s> 새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다.</s><s> 조금 늦은 나이의 도전이라 부족한 영어 실력으로 많은 어려움에 부딪쳤습니다.</s><s> 잠깐 좌절도 했지만 그보다 가장 힘들었던 것은 부모님이 주신 소중한 기회를 제대로 활용하고 있지 못하고 있는 저의 모습이었습니다.</s><s> 저는 좌절보다는 노력을 택하였습니다.'

In [15]:
tokenizer.decode(labels[0])

'새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다.</s><s> 조금 늦은 나이의 도전이라 부족한 영어 실력으로 많은 어려움에 부딪쳤습니다.</s><s> 잠깐 좌절도 했지만 그보다 가장 힘들었던 것은 부모님이 주신 소중한 기회를 제대로 활용하고 있지 못하고 있는 저의 모습이었습니다.</s><s> 저는 좌절보다는 노력을 택하였습니다.</s>'

# data generator

In [16]:
inputsTensor = tf.constant(tf.keras.preprocessing.sequence.pad_sequences(inputs, dtype='int32', padding='post', value=tokenizer.pad_token_id))
labelsTensor = tf.constant(tf.keras.preprocessing.sequence.pad_sequences(labels, dtype='int32', padding='post', value=tokenizer.pad_token_id))
inputsTensor.shape, labelsTensor.shape

(TensorShape([78373, 258]), TensorShape([78373, 258]))

# Train

In [50]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path, from_pt=True)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [18]:
gpt_model = GPT2Model("taeminlee/kogpt2")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.7.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'lm_head.weight', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.11.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [21]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
gpt_model.compile(optimizer=optimizer, loss=[loss, *[None] * gpt_model.gpt2.config.n_layer], metrics=[metric])

In [ ]:
BATCH_SIZE = 8
NUM_EPOCHS = 3
MAX_LEN = 30
VALID_SPLIT = 0.2

In [23]:
MODEL_PATH = "GPTmodel/"

checkpoint_path = os.path.join('./', MODEL_PATH, 'GPT_weights-{epoch:02d}.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
    print("{} Directory already exists\n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} Directory create complete\n".format(checkpoint_dir))

earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=5)
    
cp_callback = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1,
                              save_best_only=True, save_weights_only=True)


history = gpt_model.fit(inputsTensor, labelsTensor, 
                        epochs=NUM_EPOCHS, 
                        batch_size=BATCH_SIZE,
                        callbacks=[earlystop_callback, cp_callback],
                        validation_split=VALID_SPLIT)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (o

C:\Users\saeji\anaconda3\envs\NLP_project\lib\site-packages\tensorflow\python\framework\indexed_slices.py:431: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


9797/9797 [==============================] - 3931s 401ms/step - loss: 1.1333 - accuracy: 0.8032
Epoch 2/3
9797/9797 [==============================] - 3849s 393ms/step - loss: 1.0443 - accuracy: 0.8128
Epoch 3/3
9797/9797 [==============================] - 3844s 392ms/step - loss: 0.9867 - accuracy: 0.8200


In [34]:
gpt_model.gpt2.save_pretrained("save")

# inference

In [35]:
model = TFGPT2LMHeadModel.from_pretrained("save")
tokenizer = PreTrainedTokenizerFast.from_pretrained("taeminlee/kogpt2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at save.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [59]:
text = '<s>저희 팀은 1등을 목표로 하였기에 좋은 성능을 지닌 모델을 개발하는 것을 목표로 삼았습니다.'

In [60]:
input_ids = tokenizer.encode(text, return_tensors='tf', add_special_tokens=False)

In [54]:
gpt_model_test = GPT2Model2('save')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at save.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [56]:
gpt_model_test(input_ids)

<tf.Tensor: shape=(1, 25, 50000), dtype=float32, numpy=
array([[[  1.759507  ,   2.2879994 , -10.332263  , ...,  -6.882934  ,
          -8.639173  ,  -6.451024  ],
        [ -2.317687  ,   1.5133607 ,  -8.373779  , ...,  -6.218377  ,
          -8.261363  ,  -5.3233585 ],
        [  2.089675  ,   4.970733  , -11.279136  , ...,  -6.977225  ,
          -6.4280157 ,  -4.1832805 ],
        ...,
        [  0.17816436,   7.9227176 , -10.113467  , ...,  -4.83937   ,
          -4.036061  ,  -4.4857936 ],
        [  2.033368  ,  14.277185  , -12.22411   , ...,  -8.134983  ,
          -5.8635554 ,  -4.117525  ],
        [  1.9952654 ,  19.359085  , -11.890331  , ...,  -3.6216197 ,
          -7.192479  ,  -4.703559  ]]], dtype=float32)>

In [ ]:
output = gpt_model.generate(
        input_ids=input_ids, pad_token_id=3, do_sample=True, max_length=30)
answer = tokenizer.decode(output[0])
print(answer.replace('<s>','').replace('</s>',''))

In [58]:
output[0]

<tf.Tensor: shape=(80,), dtype=int32, numpy=
array([    0,   811, 12178,   264,  1905,  8537,   118, 32147, 48134,
       16597,  1245, 36358,  8322,   597, 47453,  1245,  2330,  1313,
        3632, 47446,  2607, 10809, 11746,   594, 47440,     1,     0,
         811,  2708,  3946,   219, 47453,   537,  5564,   294, 25644,
        2344,  5270, 22286,  8009, 47491, 26356, 47440,     1,     0,
         104,  3946,  4973, 16956,  6238,   114,  1814, 47453,   368,
       12177, 47491,   314,  2251,  4144,   276, 22846,   172,   594,
       47440,     1,     0,  2443, 25971, 37794, 47453, 25644,  2344,
        5270, 22286,   506,  6067, 47453,   811,  4883,  2164])>

In [62]:
output = model.generate(input_ids=input_ids,
                        pad_token_id=3,
                        do_sample=True,
                        max_length=200)

answer = tokenizer.decode(output[0])
print(answer.replace('<s>','').replace('</s>',''))

 저희 팀은 1등을 목표로 하였기에 좋은 성능을 지닌 모델을 개발하는 것을 목표로 삼았습니다. 그 결과, OOOOOOO를 출품하였고, 저희 팀은 OO OOOOO OXO OOO를 함께 출품하였습니다. 경쟁작 중, OOOOOO의 우수한 Benefit, 3D Finalizing 등을 비교하고, 3D Finalizing이 갖는 유용성, 친환경성 등을 조사했습니다. 그 결과, 제가 개발하였던 AutoCAD, Dragonation cell 등의 분석 기법이 우수하여, 심사위원들로부터 최우수상을 수상할 수 있었습니다. 이러한 경험을 통해, 팀원에게 공동의 목표를 달성하고, 공동의 목표 달성에 최선을 다하는 것이 팀원의 업무수행에 있어 가장 중요한 자세라는 것을 배울 수 있었습니다. 저에게 주어진 직무수행에서 책임감을 느끼고, 맡은 바를 수행하여 팀의 목표달성에 이바지하는 사원이 되겠습니다. 이를 통해, 우리회사의 미래 먹거리와


In [ ]:
df['content'][0]

['새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다. 조금 늦은 나이의 도전이라 부족한 영어 실력으로 많은 어려움에 부딪쳤습니다. 잠깐 좌절도 했지만 그보다 가장 힘들었던 것은 부모님이 주신 소중한 기회를 제대로 활용하고 있지 못하고 있는 저의 모습이었습니다. 저는 좌절보다는 노력을 택하였습니다. 매주 영어 학원과 스터디그룹에 참석하였으며 다양한 이들과의 대화를 위해 혼자 펍에 가 친구를 사귀기도 하고 아르바이트를 통해 많은 이들과의 소통으로 영어 실력뿐만 아니라 소통력과 관계력까지도 키울 수 있었습니다. 이러한 경험을 바탕으로 경동나비엔의 일원이 되어서도 어떠한 어려움 속에서도 좌절하지 않고 백절불굴의 자세로 부족한 점을 계속해서 연구하고 개선하여 끊임없는 변화 속에서 끊임없는 노력으로 성공이라는 꽃을 피우도록 하겠습니다. ',
 '코카콜라의 뉴코크 실패는 소비자는 단순히 제품만을 소비하는 것이 아니라 그 브랜드 즉, 제품의 상징성을 구매하는 심리를 보여주고 있습니다. 해외시장에 존재하는 고객은 국내시장에 존재하는 고객들보다 기업에 대한 적은 정보를 가지고 있으며 시장진출에 있어 문화, 언어적인 부분을 넘어 다양한 것들을 고려하고 새롭게 시작하여야 합니다. 하지만 이 또한 기회가 될 수 있습니다 해외시장의 경쟁자들이 보여주지 않았던 부분을 통해 시장을 공략한다면 그 부분이 기업의 상징처럼 되어 하나의 문화, 브랜드가 되어 소비자가 소비하는 이유가 될 수 있습니다. 따라서 제품 구매 후에도 빠르고 다양한 서비스를 제공하는 확실한 사후서비스를 더 강화하고 강조한 전략을 통해 해외시장을 공략하고 애프터 서비스 하면 경동나비엔이 제일 먼저 해외시장의 사람들의 뇌리에 떠오를 수 있게 상징화시킬 수 있다면 해외시장 점유율을 꾸준히 높여갈 수 있을 것입니다. ']